<a href="https://colab.research.google.com/github/ddiaz1999/Taller2_IA/blob/main/Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import prettytable
import time
import os
from IPython.display import clear_output

#import matplotlib.pyplot as plt
#import plotly.graph_objects as go
#import plotly.express as px
#import datetime
#from plotly.subplots import make_subplots

In [101]:
#/content/drive/My Drive/TALLER 2 IA/dataset
path = '/content/drive/My Drive/TALLER 2 IA/dataset'
for dirname, _, filenames in os.walk(path):
  print(dirname)
  for file in filenames:
    print(os.path.join(path,file))


path_crime = '/content/drive/My Drive/TALLER 2 IA/dataset/crime.csv'

data = pd.read_csv(path_crime,encoding = 'latin')
data.head()

/content/drive/My Drive/TALLER 2 IA/dataset
/content/drive/My Drive/TALLER 2 IA/dataset/crime.csv
/content/drive/My Drive/TALLER 2 IA/dataset/offense_codes.csv


,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [102]:
print(data.columns)

Index(['INCIDENT_NUMBER', 'OFFENSE_CODE', 'OFFENSE_CODE_GROUP',
       'OFFENSE_DESCRIPTION', 'DISTRICT', 'REPORTING_AREA', 'SHOOTING',
       'OCCURRED_ON_DATE', 'YEAR', 'MONTH', 'DAY_OF_WEEK', 'HOUR', 'UCR_PART',
       'STREET', 'Lat', 'Long', 'Location'],
      dtype='object')


In [103]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns 
import folium 
from folium.plugins import HeatMap
from fbprophet import Prophet


data = data.rename(columns={'OFFENSE_CODE':'Code','OFFENSE_CODE_GROUP':'Group','OFFENSE_DESCRIPTION':'Description','OCCURRED_ON_DATE':'Date'})
data.head()
#print(data.shape)

,INCIDENT_NUMBER,Code,Group,Description,DISTRICT,REPORTING_AREA,SHOOTING,Date,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [104]:
data.drop(['INCIDENT_NUMBER','Code','SHOOTING','UCR_PART','Lat','Long','Location'],inplace=True,axis=1)
data.head()

,Group,Description,DISTRICT,REPORTING_AREA,Date,YEAR,MONTH,DAY_OF_WEEK,HOUR,STREET
0,Larceny,LARCENY ALL OTHERS,D14,808,2018-09-02 13:00:00,2018,9,Sunday,13,LINCOLN ST
1,Vandalism,VANDALISM,C11,347,2018-08-21 00:00:00,2018,8,Tuesday,0,HECLA ST
2,Towed,TOWED MOTOR VEHICLE,D4,151,2018-09-03 19:27:00,2018,9,Monday,19,CAZENOVE ST
3,Investigate Property,INVESTIGATE PROPERTY,D4,272,2018-09-03 21:16:00,2018,9,Monday,21,NEWCOMB ST
4,Investigate Property,INVESTIGATE PROPERTY,B3,421,2018-09-03 21:05:00,2018,9,Monday,21,DELHI ST


In [105]:
data['Date']=pd.to_datetime(data['Date'])
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319073 entries, 0 to 319072
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Group           319073 non-null  object        
 1   Description     319073 non-null  object        
 2   DISTRICT        317308 non-null  object        
 3   REPORTING_AREA  319073 non-null  object        
 4   Date            319073 non-null  datetime64[ns]
 5   YEAR            319073 non-null  int64         
 6   MONTH           319073 non-null  int64         
 7   DAY_OF_WEEK     319073 non-null  object        
 8   HOUR            319073 non-null  int64         
 9   STREET          308202 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 24.3+ MB


Motor Vehicle Accident Response    37132
Larceny                            25935
Medical Assistance                 23540
Investigate Person                 18750
Other                              18075
Drug Violation                     16548
Simple Assault                     15826
Vandalism                          15415
Verbal Disputes                    13099
Towed                              11287
Investigate Property               11124
Larceny From Motor Vehicle         10847
Property Lost                       9751
Warrant Arrests                     8407
Aggravated Assault                  7807
Name: Group, dtype: int64


In [106]:
data.index=pd.DatetimeIndex(data.Date)
data.head()
order_data1=data['Group'].value_counts().iloc[:15]
print(order_data1)
order_data=data['Group'].value_counts().iloc[:15].index
plt.figure(figsize=(15,10))
sns.countplot(y='Group',data=data,order=order_data)
plt.ioff()

In [107]:
crimes = data['Group'].value_counts().iloc[:15]

crimes = list(crimes.keys())

data = data.loc[data['Group'].isin(crimes)]

data['Group'] = data['Group'].astype('category')
data = pd.get_dummies(data=data['Group'])

data.head()

,Aggravated Assault,Drug Violation,Investigate Person,Investigate Property,Larceny,Larceny From Motor Vehicle,Medical Assistance,Motor Vehicle Accident Response,Other,Property Lost,Simple Assault,Towed,Vandalism,Verbal Disputes,Warrant Arrests
Date,,,,,,,,,,,,,,,
2018-09-02 13:00:00,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2018-08-21 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2018-09-03 19:27:00,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2018-09-03 21:16:00,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2018-09-03 21:05:00,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [108]:
data = data.resample('M').sum()
data.head()
#print(data)

,Aggravated Assault,Drug Violation,Investigate Person,Investigate Property,Larceny,Larceny From Motor Vehicle,Medical Assistance,Motor Vehicle Accident Response,Other,Property Lost,Simple Assault,Towed,Vandalism,Verbal Disputes,Warrant Arrests
Date,,,,,,,,,,,,,,,
2015-06-30,124.0,218.0,189.0,104.0,446.0,170.0,226.0,424.0,297.0,127.0,201.0,110.0,210.0,112.0,101.0
2015-07-31,245.0,545.0,355.0,191.0,826.0,324.0,468.0,870.0,502.0,254.0,450.0,194.0,406.0,253.0,209.0
2015-08-31,221.0,488.0,354.0,180.0,773.0,340.0,427.0,910.0,563.0,258.0,409.0,244.0,436.0,223.0,207.0
2015-09-30,234.0,527.0,387.0,202.0,743.0,290.0,498.0,941.0,550.0,246.0,381.0,282.0,404.0,217.0,215.0
2015-10-31,172.0,590.0,385.0,181.0,769.0,363.0,490.0,927.0,512.0,234.0,397.0,253.0,446.0,205.0,209.0


In [109]:
len_train = int(data.shape[0] * 0.85)

X = np.array(data.index)
X = X[:,np.newaxis]
X_train = X[0:len_train]
X_test = X[len_train:len(X)]
print(X.shape, X_train.shape, X_test.shape)

(40, 1) (34, 1) (6, 1)


In [110]:
Y_train = []
y_test = []
for crime in crimes:
  y = data[crime]
  Y_train.append(y[0:len_train])
  y_test.append(y[len_train:len(y)])

Y_train = np.array(Y_train)
y_test = np.array(y_test)

print(Y_train.shape, y_test.shape)

(15, 34) (15, 6)


In [111]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR

def train(X_train, Y_train):
  regr = {}
  for idx, crime in enumerate(Y_train):
    regr[crimes[idx]] = make_pipeline(MinMaxScaler(), SVR())
    regr[crimes[idx]].fit(X_train, crime)
  return regr

predictors = train(X_train, Y_train)
print(len(predictors))

15


In [112]:
def evaluate_model(estimators, X_test, y_test):
  scores = []
  for idx in range(len(estimators)):
    scores.append(estimators[crimes[idx]].score(X_test, y_test[idx]))
  global_score = sum(scores)/len(scores)
  
  print('Global Score was: ', global_score)

evaluate_model(predictors, X_test, y_test)

Global Score was:  -0.35421460128044574


In [151]:
import collections
from prettytable import PrettyTable

def predict_crime(estimators, X_test, y_test):
  Y_predict = []
  for idx, predictors in enumerate(estimators):
    Y_predict.append(estimators[crimes[idx]].predict(X_test))
  Y_predict = np.array(Y_predict).T

  predict_df = show_predict(X_test, Y_predict)
  return predict_df

def show_predict(X, Y):

  predict_frame = collections.defaultdict(list)
  for i in range(len(X)):
    predict_frame['Date'].append(X[i])
    for j, crime in enumerate(crimes):
      predict_frame[crime].append(Y[i][j])
  
  predict_frame = pd.DataFrame(predict_frame)

  return predict_frame

m = predict_crime(predictors, X_test, y_test)
m.head()

,Date,Motor Vehicle Accident Response,Larceny,Medical Assistance,Investigate Person,Other,Drug Violation,Simple Assault,Vandalism,Verbal Disputes,Towed,Investigate Property,Larceny From Motor Vehicle,Property Lost,Warrant Arrests,Aggravated Assault
0,[2018-04-30T00:00:00.000000000],978.861592,667.936442,593.941580,438.200087,461.823028,413.244550,404.407968,401.497628,349.215932,292.584637,291.635520,275.468411,250.515180,223.587709,187.242227
1,[2018-05-31T00:00:00.000000000],978.350620,668.043672,593.256450,437.622362,461.944977,413.669934,404.046757,401.522069,348.792509,291.922280,291.078514,275.700594,250.266872,223.104935,187.236282
2,[2018-06-30T00:00:00.000000000],977.888723,668.191446,592.595586,437.149273,462.058605,414.121062,403.715883,401.624670,348.430952,291.333252,290.576880,275.957358,250.062466,222.753129,187.291695
3,[2018-07-31T00:00:00.000000000],977.453463,668.383899,591.934927,436.753101,462.176555,414.611301,403.406361,401.798798,348.108015,290.788935,290.107264,276.247618,249.888043,222.501510,187.405552
4,[2018-08-31T00:00:00.000000000],977.066041,668.607688,591.313277,436.448746,462.297502,415.108270,403.138342,402.025209,347.834923,290.315503,289.693414,276.553301,249.748427,222.350731,187.566383


In [150]:
n = show_predict(X_test, y_test.T)
n.head()

,Date,Motor Vehicle Accident Response,Larceny,Medical Assistance,Investigate Person,Other,Drug Violation,Simple Assault,Vandalism,Verbal Disputes,Towed,Investigate Property,Larceny From Motor Vehicle,Property Lost,Warrant Arrests,Aggravated Assault
0,[2018-04-30T00:00:00.000000000],878.0,613.0,724.0,425.0,449.0,388.0,438.0,281.0,346.0,320.0,314.0,220.0,280.0,143.0,198.0
1,[2018-05-31T00:00:00.000000000],990.0,681.0,807.0,494.0,541.0,411.0,553.0,394.0,402.0,365.0,359.0,261.0,308.0,155.0,245.0
2,[2018-06-30T00:00:00.000000000],1063.0,671.0,790.0,426.0,450.0,422.0,512.0,404.0,394.0,363.0,373.0,301.0,318.0,137.0,240.0
3,[2018-07-31T00:00:00.000000000],964.0,790.0,676.0,521.0,460.0,435.0,469.0,352.0,407.0,298.0,325.0,282.0,283.0,123.0,233.0
4,[2018-08-31T00:00:00.000000000],1000.0,712.0,640.0,646.0,447.0,359.0,404.0,391.0,419.0,379.0,292.0,317.0,308.0,154.0,178.0


In [ ]:
'''
X_test = np.array(pd.date_range(start='10/10/2018', periods=2, freq='2Y'))
X_test = X_test[:,np.newaxis]
print(X_test)
print(X_test.shape)
'''